In [45]:
 from google.colab import drive
 import os

# README - Execute this cell to mount the notebook in your google drive. 
# Execute the cell and follow the link to sign and, paste the given key in the little text box. The credentials are only available for you. 

drive.mount('/content/drive', force_remount=True)

if not os.path.exists("/content/drive/MyDrive/audio-dataset"):
  print("Pulling dataset")
  os.mkdir("/content/drive/MyDrive/audio-dataset")
  !git clone https://github.com/emilstahl97/Audio-dataset.git
else:
  print("Dataset already exists")

#os.chdir("/content/drive/MyDrive/audio-dataset/Audio-dataset/Audio-dataset")
os.chdir("/content/drive/MyDrive/audio-dataset/Audio-dataset/Rawdata")

!git pull
!ls

RAVDESS_PATH = "./RAVDESS"
SAVEE_PATH = "./SAVEE"



Mounted at /content/drive
Dataset already exists
Already up to date.
03-01-01-01-01-01-01.wav  03-01-04-01-01-01-01.wav  03-01-06-02-01-01-01.wav
03-01-01-01-01-01-02.wav  03-01-04-01-01-01-02.wav  03-01-06-02-01-01-02.wav
03-01-01-01-01-01-03.wav  03-01-04-01-01-01-03.wav  03-01-06-02-01-01-03.wav
03-01-01-01-01-01-04.wav  03-01-04-01-01-01-04.wav  03-01-06-02-01-01-04.wav
03-01-01-01-01-01-06.wav  03-01-04-01-01-01-06.wav  03-01-06-02-01-01-06.wav
03-01-01-01-01-01-07.wav  03-01-04-01-01-01-07.wav  03-01-06-02-01-01-07.wav
03-01-01-01-01-01-08.wav  03-01-04-01-01-01-08.wav  03-01-06-02-01-01-08.wav
03-01-01-01-01-01-09.wav  03-01-04-01-01-01-09.wav  03-01-06-02-01-01-09.wav
03-01-01-01-01-01-10.wav  03-01-04-01-01-01-10.wav  03-01-06-02-01-01-10.wav
03-01-01-01-01-01-11.wav  03-01-04-01-01-01-11.wav  03-01-06-02-01-01-11.wav
03-01-01-01-01-01-12.wav  03-01-04-01-01-01-12.wav  03-01-06-02-01-01-12.wav
03-01-01-01-01-01-13.wav  03-01-04-01-01-01-13.wav  03-01-06-02-01-01-13.wav
03-01-0

##**PIP**

In [46]:
!pip install --upgrade tensorflow_hub
!pip install --upgrade tensorflow
!pip install --upgrade keras
!pip install --upgrade numpy
!pip install --upgrade matplotlib
!pip install --upgrade librosa
!pip install --upgrade scipy
!pip install --upgrade scikit-learn
!pip install --upgrade pandas

##**IMPORTING**

In [47]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import scipy.io.wavfile
import numpy as np
import pandas as pd
import sys

print("Everything imported")

Everything imported


## **Make list**

In [48]:
mylist= os.listdir('./')
type(mylist)
print("Number of audio files: {}".format(len(mylist)))

print(mylist[1][6:-16])



Number of audio files: 1380
01


##**Get labels**

In [51]:
feeling_list=[]
for item in mylist:
    if item[6:-16]=='02' and int(item[18:-4])%2==0:
        feeling_list.append('female_calm')
    elif item[6:-16]=='02' and int(item[18:-4])%2==1:
        feeling_list.append('male_calm')
    elif item[6:-16]=='03' and int(item[18:-4])%2==0:
        feeling_list.append('female_happy')
    elif item[6:-16]=='03' and int(item[18:-4])%2==1:
        feeling_list.append('male_happy')
    elif item[6:-16]=='04' and int(item[18:-4])%2==0:
        feeling_list.append('female_sad')
    elif item[6:-16]=='04' and int(item[18:-4])%2==1:
        feeling_list.append('male_sad')
    elif item[6:-16]=='05' and int(item[18:-4])%2==0:
        feeling_list.append('female_angry')
    elif item[6:-16]=='05' and int(item[18:-4])%2==1:
        feeling_list.append('male_angry')
    elif item[6:-16]=='06' and int(item[18:-4])%2==0:
        feeling_list.append('female_fearful')
    elif item[6:-16]=='06' and int(item[18:-4])%2==1:
        feeling_list.append('male_fearful')
    elif item[:1]=='a':
        feeling_list.append('male_angry')
    elif item[:1]=='f':
        feeling_list.append('male_fearful')
    elif item[:1]=='h':
        feeling_list.append('male_happy')
    #elif item[:1]=='n':
        #feeling_list.append('neutral')
    elif item[:2]=='sa':
        feeling_list.append('male_sad')


#print(feeling_list)
#labels = pd.DataFrame(feeling_list)
#labels[:10]

## Getting the features of audio files using librosa

In [53]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    if mylist[index][6:-16]!='01' and mylist[index][6:-16]!='07' and mylist[index][6:-16]!='08' and mylist[index][:2]!='su' and mylist[index][:1]!='n' and mylist[index][:1]!='d':
        X, sample_rate = librosa.load('./'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1



#df[:5]
#df3 = pd.DataFrame(df['feature'].values.tolist())
#df3[:5]

In [61]:
df3 = pd.DataFrame(df['feature'].values.tolist())

newdf = pd.concat([df3,labels], axis=1)

rnewdf = newdf.rename(index=str, columns={"0": "label"})

print(rnewdf)

from sklearn.utils import shuffle
rnewdf = shuffle(newdf)

rnewdf=rnewdf.fillna(0)

           0          1          2    ...        214        215             0  
0   -70.267769 -70.267769 -70.267769  ... -61.082737 -60.234661       male_calm
1   -67.557388 -67.557388 -67.557388  ... -66.086418 -66.205048       male_calm
2   -69.673286 -69.693306 -69.693306  ... -62.603935 -61.121773       male_calm
3   -69.051399 -69.051399 -69.051399  ... -67.008553 -68.636299       male_calm
4   -73.841370 -73.841370 -73.841370  ... -57.508217 -56.577332       male_calm
..         ...        ...        ...  ...        ...        ...             ...
915 -31.163713 -28.617668 -24.917967  ... -36.526012 -36.526012  female_fearful
916 -51.839401 -51.839401 -51.839401  ... -44.039024 -44.126728    male_fearful
917 -49.289055 -49.289055 -49.048580  ... -49.057945 -48.911900  female_fearful
918 -57.722847 -57.722847 -57.722847  ... -50.740566 -51.029232    male_fearful
919 -45.345825 -45.345825 -45.345825  ... -44.287987 -44.375999  female_fearful

[920 rows x 217 columns]


## Dividing the data into test and train

In [66]:
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]

print(train)

           0          1          2    ...        214        215             0  
714 -46.277058 -46.441635 -47.998329  ... -45.505005 -45.232563      male_angry
47  -50.268387 -51.055355 -48.980633  ... -46.921928 -49.940742     female_calm
598 -53.901146 -52.951214 -53.601170  ... -50.363785 -52.028522      male_angry
765 -49.850674 -50.502468 -50.002644  ... -26.224455 -25.959602  female_fearful
87  -59.765766 -59.400639 -59.620049  ... -46.223816 -43.273678     female_calm
..         ...        ...        ...  ...        ...        ...             ...
6   -73.254967 -73.254967 -73.254967  ... -24.115793 -21.873394       male_calm
617 -50.773075 -51.290535 -49.537144  ... -50.153622 -50.676296    female_angry
145 -79.309151 -79.309151 -79.309151  ... -66.494026 -66.106949     female_calm
786 -63.942562 -65.589714 -66.062729  ... -64.663841 -66.025917    male_fearful
179 -47.387753 -48.970577 -52.059589  ... -46.134380 -45.552658     female_calm

[743 rows x 217 columns]


In [68]:
trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]
testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]

In [69]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [70]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [71]:
X_train.shape

(743, 216)

# Padding sequence for CNN model

In [72]:
print('Pad sequences')
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

Pad sequences


In [80]:
import tensorflow

model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(216,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))
opt = tensorflow.keras.optimizers.RMSprop(learning_rate =0.00001, decay=1e-6)

In [81]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_42 (Conv1D)          (None, 216, 128)          768       
                                                                 
 activation_49 (Activation)  (None, 216, 128)          0         
                                                                 
 conv1d_43 (Conv1D)          (None, 216, 128)          82048     
                                                                 
 activation_50 (Activation)  (None, 216, 128)          0         
                                                                 
 dropout_14 (Dropout)        (None, 216, 128)          0         
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 27, 128)          0         
 1D)                                                             
                                                      

In [82]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [ ]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=32, epochs=1009, validation_data=(x_testcnn, y_test))

Epoch 1/1009
24/24 [==============================] - 5s 154ms/step - loss: 2.3816 - accuracy: 0.1117 - val_loss: 2.2939 - val_accuracy: 0.1638
Epoch 2/1009
24/24 [==============================] - 3s 143ms/step - loss: 2.3218 - accuracy: 0.0929 - val_loss: 2.2909 - val_accuracy: 0.1582
Epoch 3/1009
24/24 [==============================] - 3s 141ms/step - loss: 2.2970 - accuracy: 0.1211 - val_loss: 2.2811 - val_accuracy: 0.1356
Epoch 4/1009
24/24 [==============================] - 3s 141ms/step - loss: 2.2902 - accuracy: 0.1225 - val_loss: 2.2670 - val_accuracy: 0.1130
Epoch 5/1009
24/24 [==============================] - 3s 141ms/step - loss: 2.2680 - accuracy: 0.1467 - val_loss: 2.2551 - val_accuracy: 0.0960
Epoch 6/1009
24/24 [==============================] - 3s 141ms/step - loss: 2.2605 - accuracy: 0.1440 - val_loss: 2.2542 - val_accuracy: 0.1073
Epoch 7/1009
24/24 [==============================] - 3s 143ms/step - loss: 2.2566 - accuracy: 0.1252 - val_loss: 2.2447 - val_accuracy:

## Plotting the accuracy and loss graph

In [ ]:
#sigmoid
plt.plot(cnnhistory.history['acc'])
plt.plot(cnnhistory.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()